In [ ]:
pip install langdetect

     |████████████████████████████████| 981 kB 7.4 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=c9fd52fd28f79740d0d1c758727bf38dcc36a4b977044768ba34eaf4f7b87df1
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
from wordcloud import WordCloud
from keras.backend import manual_variable_initialization 
manual_variable_initialization(True)

import nltk
import spacy
nlp=spacy.load("en_core_web_sm")
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

# Twitter library
import tweepy
import requests
import json
import tensorflow
from numpy.random import seed
seed(0)
tensorflow.random.set_seed(0)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
data=pd.read_csv("https://raw.githubusercontent.com/aayushpandey1703/fyp-project/main/AnnotatedData3.csv")
data.head()

,class,text
0,yes,i'll buy the iphone x if it can get ac...
1,yes,\twant sugarmommy buy iphone 13
2,yes,\twant sugarmommy buy iphone 12
3,yes,i want buy iphone someone please hook reliable...
4,no,Some dude in FB selling the iPhone X 64 gb...


## Exploratory data analysis

In [ ]:
data.shape
data.isnull().sum()

class    1
text     1
dtype: int64

## Data preprocessing

In [ ]:
data['class']=data['class'].replace(['Undefined','undefined'],'un')
data=data[data['class']!='un']
data.head(10)

,class,text
0,yes,i'll buy the iphone x if it can get ac...
1,yes,\twant sugarmommy buy iphone 13
2,yes,\twant sugarmommy buy iphone 12
3,yes,i want buy iphone someone please hook reliable...
4,no,Some dude in FB selling the iPhone X 64 gb...
5,yes,Home dab emote man today and I get hopped on...
6,yes,Buy an iPhone X
7,no,Brother I don't have iPhone or Android 11.0 I ...
8,no,I hate iOS 11. My iPhone 6+ works 10X slower...
11,no,Bo-go sale tomorrow at T-Mobile buy one get...


In [ ]:
data['class']=data['class'].apply(lambda x: 1 if x=='yes' else 0)
data.head(10)

,class,text
0,1,i'll buy the iphone x if it can get ac...
1,1,\twant sugarmommy buy iphone 13
2,1,\twant sugarmommy buy iphone 12
3,1,i want buy iphone someone please hook reliable...
4,0,Some dude in FB selling the iPhone X 64 gb...
5,1,Home dab emote man today and I get hopped on...
6,1,Buy an iPhone X
7,0,Brother I don't have iPhone or Android 11.0 I ...
8,0,I hate iOS 11. My iPhone 6+ works 10X slower...
11,0,Bo-go sale tomorrow at T-Mobile buy one get...


In [ ]:
data.shape

(1574, 2)

# Text Processing

In [ ]:
Stopwords=set(stopwords.words('english'))
emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  
                               u"\U0001F300-\U0001F5FF"  
                               u"\U0001F680-\U0001F6FF"  
                               u"\U0001F1E0-\U0001F1FF"  
                               u"\U00002500-\U00002BEF"  
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  
                               u"\u3030"
                               "]+", flags=re.UNICODE)


In [ ]:
def clean_text(text):
  text=" ".join(word for word in str(text).split() if word not in Stopwords) # remove stopwords
  text=text.lower()                                                     #converting to lower
  text=re.sub(r'https?:\/\/\S+', '', text)                              #removing hyperlinks
  text=re.sub(r'@[a-zA-Z0-9_@./#&+-]+', '', text)                       #removing tags
  text=re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', text)          #removing web links
  text= re.sub(r'&[a-z]+;', '', text)                                   #removing html reference characters
  text=re.sub(r'\n', ' ', text)
  text=re.sub(r"(.)\1\1+",r"\1", text)                                  #removing repeated letters
  text=re.sub(r'\b(\w+)(\s+\1)+\b',r'\1', text)                         #removing repeated words
  text=emoji_pattern.sub('',text)                                       #remove emoticons
  doc,text=nlp(text),''                                                 # remove currency and punctuations
  for token in doc:
    if token.is_currency!=True and token.ent_type_!="MONEY" and token.is_punct!=True:
      text+=" "+str(token)
    
  return text
  
data['text']=data['text'].apply(clean_text)
data.head(10)

KeyboardInterrupt: ignored

## Vectorization

In [ ]:
t=Tokenizer(num_words=50000,lower=True)
t.fit_on_texts(data['text'])
x=t.texts_to_sequences(data['text'])
x=sequence.pad_sequences(x,maxlen=250)
print(x.shape)
print(x)

In [ ]:
file_to_store = open("/content/drive/MyDrive/stored_object.pickle", "wb")
pickle.dump(data['text'], file_to_store)
file_to_store.close()


In [ ]:
file_to_read = open("/content/drive/MyDrive/stored_object.pickle", "rb")
z=pickle.load(file_to_read)
z

# calculate sentiment and subjectivity of tweets
#### subjectivity quantifies the amount of personal opinion and factual information contained in the text. The higher subjectivity means that the text contains personal opinion rather than factual information. TextBlob has one more parameter — intensity. TextBlob calculates subjectivity by looking at the ‘intensity’. Intensity determines if a word modifies the next word. For English, adverbs are used as modifiers (‘very good’).  Subjectivity lies between [0,1].

#### Polarity lies between [-1,1], -1 defines a negative sentiment and 1 defines a positive sentiment. Negation words reverse the polarity

In [ ]:
def sentiment_analysis(text):
    return TextBlob(text).sentiment.polarity
def subjectivity_analysis(text):
    return TextBlob(text).sentiment.subjectivity
data_sample=data
data_sample['polarity']=data_sample['text'].apply(sentiment_analysis)
data_sample['subjectivity']=data_sample['text'].apply(subjectivity_analysis)
data_sample

In [ ]:
def count_words(text):
    l=text.split()
    return len(l)
data_sample['word_count']=data_sample['text'].apply(count_words)
data_sample

# Data Visualization

In [ ]:
plt.rcParams['figure.figsize']=(15,8)
f,axes=plt.subplots(1,2)
sns.countplot(data['class'],ax=axes[0])
sns.histplot(data,x='word_count',binwidth=5,kde=True)

In [ ]:
plt.rcParams['figure.figsize']=(15,8)
f,axes=plt.subplots(1,2)
sns.histplot(data=data_sample,x='subjectivity',element="poly",ax=axes[0])
sns.histplot(data=data_sample,x='polarity',element="poly")

In [ ]:
# visualize most recuring words in purchase intent and non purchase intent tweets
data_pi=data[data['class']==1]
data_no=data[data['class']==0]
wc=WordCloud(background_color='white',width=1600, height=800)
wc.generate(" ".join(data_pi['text']))
wc2=WordCloud(background_color='white',width=1600, height=800)
wc2.generate(" ".join(data_no['text']))
plt.imshow(wc)

In [ ]:
plt.imshow(wc2)

#LSTM model

In [ ]:
y=pd.get_dummies(data['class']).values
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1)
print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)
y

## training model

In [ ]:
model=Sequential()
model.add(Embedding(50000,100,input_length=x.shape[1]))
# model.add(SpatialDropout1D(0.2))
model.add(LSTM(100,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
fit=model.fit(x_train,y_train,epochs=5,batch_size=32,validation_split=0.1)


In [ ]:
model.get_weights()

In [ ]:
model.save_model("/content/drive/MyDrive/modeln.h5")

In [ ]:
import hickle as hkl

hkl.dump(model, 'test.hkl', mode='w')

## Analyze model performance

In [ ]:
import tensorflow as tf

model.save('/content/drive/MyDrive/64x3-CNN.model')


In [ ]:
acr=model.evaluate(x_test,y_test)
model.summary()

In [ ]:
cal=sequence.pad_sequences(x,maxlen=250)
pred=model.predict(x_test)
pred=np.argmax(pred,axis=1)
pred=np.reshape(pred,(len(pred),1))


In [ ]:
y=list(y_test)
y2=[]
for i in range(len(y)):
    if y[i][0]==1:
        y2.append(0)
    else:
        y2.append(1)
len(y2)

In [ ]:
x=t.texts_to_sequences(["I hate Iphone"]) # pro 
x=sequence.pad_sequences(x,maxlen=250)
p=model.predict(x)
print(p)
pred=np.argmax(p,axis=1)
if pred[0]==1:
  print('yes')
else:
  print('no')

In [ ]:
x=t.texts_to_sequences(["People saying iOS is more user friendly and easy to use must have never had an Android. I bought my first iPhone 6 months ago and I hate it. Siri is dumb AF, I cannot download a fckin PDF from my browser, I need to press 5 things for something I would need just 1 on Android"])
x=sequence.pad_sequences(x,maxlen=250)
p=model.predict(x)
pred=np.argmax(p,axis=1)
if pred[0]==1:
  print('yes')
else:
  print('no')

# Scrape Tweets using tweepy

In [ ]:
consumer_api='vtpCktz8FTXUKEqbrmQyfitZ2'
consumer_api_secret='FDbBJ3gTQPvv2lA9jT23DDbnDJipV4NjmfEZMobi8Xl7zJO2L1'
access_token='1287085799339462656-yMJ0c9NzCgbwu7sd9QEVjyXUKkWieS'
access_secret='OPq0IiDTyGmhziS0EUNyYbupdr492tuB848rtGT8BxzLS'
bearer_token='AAAAAAAAAAAAAAAAAAAAAJbiNgEAAAAA9XfDS3YRCsEWv9GoitFHFTCuu9I%3DFdYzAbd2GvMgStnVTDVlnD1TVMARk4w306nGUt9YHNoT2AkUPQ'

In [ ]:
def search_twitter(query, tweet_fields, bearer_token):
  headers = {"Authorization": "Bearer {}".format(bearer_token)}

  url = "https://api.twitter.com/2/tweets/search/recent?&query={}&tweet.fields={}&user.fields=name,created_at&max_results=100&expansions=author_id".format(query, tweet_fields )
  response = requests.request("GET", url, headers=headers)

  return response.json()

query="lang%3Aen%20i want to buy iphone -is:retweet OR i don't love iphone -is:retweet"
tweet_fields='text,author_id,created_at,lang'
json_response = search_twitter(query=query, tweet_fields=tweet_fields, bearer_token=bearer_token)

#pretty printing
print(len(json_response['data']))
print(len(json_response['includes']['users']))
print(json.dumps(json_response,indent=4))

## Normalize scraped tweets
removing tweets with no username or usernames with no tweets

In [ ]:
data=[]
for i in range(min(len(json_response['data']),len(json_response['includes']['users']))):
  text=json_response['data'][i]['text']
  print(detect(text))
  if detect(text)!='en' :
    continue
  username=json_response['includes']['users'][i]['username']
  data.append({'username':username,'text':text})
data_obj={'data':data}

print(json.dumps(data_obj,indent=4))


In [ ]:
p=pd.DataFrame(data=data)
p['text'] = p.text.apply(lambda x: re.sub(r'&[a-z]+;', '', x))
p['text'] = p.text.apply(lambda x: re.sub(r'\\u[a-z0-9]+', '', x))
p

# scraped tweet pre-processing

## Analyzing single text

In [ ]:
tweet_description=[]
doc=nlp("@nines1128 dude fb selling apple iphone 12 for $ 1100 like nigga one gon na buy shit get 256 gb price")
for token in doc:
  a={'token':token,nlp.pipe_names[0]:token.pos_,nlp.pipe_names[1]:token.dep_,nlp.pipe_names[2]:token.ent_type_}
  tweet_description.append(a)

tweet_description_dataframe=pd.DataFrame(data=tweet_description)
tweet_description_dataframe


## preparing tweets dataframe 


In [ ]:
tweet_dataframe=pd.DataFrame(data=data)
tweet_dataframe

## Processing scraped tweets
Using clean_text function

In [ ]:
tweet_dataframe['text']=tweet_dataframe['text'].apply(clean_text)
tweet_dataframe                                                     # final processed tweets dataframe

# Analyzing model performance on scraped tweets

In [ ]:
tweet_dataframe

In [ ]:
aa=tweet_dataframe
aa=pd.Series(aa.text.values,index=aa.username).to_dict()
a=[]
ps=dict()
for i in aa:
  x=t.texts_to_sequences([aa[i]])
  x=sequence.pad_sequences(x,maxlen=250)
  p=model.predict(x)
  pred=np.argmax(p,axis=1)
  p=np.reshape(pred,(len(pred),1))
  if p==1:
    a.append('yes')
    ps[i]=aa[i]
  else:
    a.append('no')
x=a.count('yes')
y=a.count('no')

In [ ]:
sentiment = ['Yes', 'No']
data = [x, y]
explode = (0.1, 0.0)
colors = ( "cyan", "orange")
wp = { 'linewidth' : 2, 'edgecolor' : "blue" }
def func(pct, allvalues):
    absolute = int(pct / 100.*np.sum(allvalues))
    return "{:.1f}%\n({:d})".format(pct, absolute)
 
fig, ax = plt.subplots(figsize =(10, 7))
wedges, texts, autotexts = ax.pie(data,
                                  autopct = lambda pct: func(pct, data),
                                  explode = explode,
                                  labels = sentiment,
                                  shadow = True,
                                  colors = colors,
                                  startangle = 90,
                                  wedgeprops = wp,
                                  textprops = dict(color ="black"))
 
ax.legend(wedges, sentiment,
          title ="Sentiment",
          loc ="center left",
          bbox_to_anchor =(1, 0, 1, 1))
 
plt.setp(autotexts, size = 10, weight ='bold')
ax.set_title("Sentimental Values")
 
plt.show()
print(ps)

# Save model

In [ ]:
from keras.models import load_model

In [ ]:
import pickle
filename = '/content/drive/MyDrive/finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
fd=pickle.load(open(filename, 'rb'))

In [ ]:
x=t.texts_to_sequences(['my sit down bruce willis i do n’t even remember movie fell love iphone mic attachment in middle interview asked assistant go pick one up he like kid very real excited that ’s i remember', '   i respectfully disagree bitcoin digital gold   ust digital cash i want buy iphone   stock',' that u put google photos cuz iphone gallery i know ur ex got dumped cuz always love get that ’s happened u lie ppl i coming u do n’t care ur girl',' loved tip too make ig ads look simplistic native possible do n’t go super stylized 15 secs less personally i love amateur style ads shot iphone   look sound like product review','   i want buy iphone','   i want buy iphone','   i want buy iphone','   i want buy iphone','    i want buy iphone',' well i break buy new iphone the twitter app version i longer shows comments error message my iphone 6 perfectly good otherwise way i want twitter w/o comments','   i use encounter+ mac iphone plan i get ipad one time buy cheap subscription local hosting local network abroad want pretty easy work bit curve it ’s super helpful dm',' if want sugar daddy buy iphone buy nicki minaj cases blouses i send pictures i still live you',' someone send cash want buy iphone xr already', 'winsforsure' ' i want need new iphone i do n’t want buy money', 'LannyTv' '   i want buy iphone gib quick alpha', 'callmeshazzam' '   i do n’t know i ’m helping mansplaining here   iphone ’s photo edit mirror symmetry switch option photo edit love shirt  ', 'bigwin_metawin' '   hi chita hope win u. i want buy new phone kinda old ca nt update twitr bec using iphone 6 thankyou', 'slaytalia4pres' " okay i changed mind it 's iphone 12 i want buy 430k", 'StrawberryDiar3' '   i want buy i goinh use iphone business', 'RRichPhotos' ' do n’t get samsung phone get iphone oh i love iphone', 'myfriendblondie' ' someone buy nft i want buy iphone nft', 'mrdrew1988' ' miss i do n’t even self follow every day i thank god people i follow like post could show feed hear i love iphone', 'Shkrepsja' ' might tell dad want use money scholarship buy new iphone se', 'AceclothingNg' ' i love   joules oh do n’t notifications appear iphone very frustrating called smart device joule chefsteps', 'HarpSilver' '   perfect color iphone me i want buy it subscribe', 'folyqe' '    i do n’t what ’s happening samsung think want people buy iphone i really miss samsung note 4 7 days a company it ’s thing daring innovative ugly copycat', '__phorcys' '   good morning please i want buy iphone fast charger', 'itsmearrunima' "    i used gphotos since back pixel 2 days i love luckily i need g1 yet i 'm 5 gb i 've used gdrive forever also installed iphone 12 makes sharing easier say", 'TruvybeR' ' speaking i m gon na look iphone 12/13 mini prices want buy one', 'official_ganja' ' buy laptop want iphone please stop purchasing hahhaahahhahahahahaha', '_MrJasonW' '    thanks love android user know thing iphone', 'nofixedrange' ' i do n’t think oneplus10pro bcz i already use iphone since 2015   i love iphones', 'Nova_Lee4444' '   put someone never wants rent iphone buy iphone installments please let continue buy phone outright day 1 keep long i want', 'JustAGalAndADog' ' like iphone trending shut love huawei', 'danielrao007' '   i want buy iphone xr 170k meettolaawoska', 'chasejoynbliss' ' watching bbl videos considering vlogging hm i love talking lol sometimes little much hm decisions decisions i wonder i ’ll need camera cause pictures i take iphone do n’t give it ’s supposed give', 'f2fmuti' '   i would love try iphone any plans android pls', 'wheniminurarms' '   i iphone   another way listen show i would love 2 support you  ', '00ocIock_' '   i want buy iphone 13 dreams', 'eddypham' "   that 's i love iphone 13 mini i need anything larger", 'iam_Ozymandias_' ' want buy new iphone airpods daddy yankee tour tickets kpop albums tom ford ’s lost cherry i ’m fucking poor']) # pro 
x=sequence.pad_sequences(x,maxlen=250)
p=fd.predict(x)
print(p)
pred=np.argmax(p,axis=1)
print(pred)
if pred[0]==1:
  print('yes')
else:
  print('no')

In [ ]:
!pip install streamlit
!pip install pyngrok

In [ ]:
%%writefile app.py
import streamlit as st
st.set_page_config(layout="wide",menu_items=None)
st.title("Twitter Sentiment Analyser")
col1,col2 = st.columns(2)

option = col1.selectbox(
     'Select Brand Name',
     ('Iphone', 'Samsung', 'Xiaomi'))
col1.title("Brand Info")
old= col2.select_slider('Select how old tweets can be (in days)',options=['1', '2', '3', '4', '5', '6', '7'])
colx1,colx2,colx3,colx4=st.columns(4)
colx1.image("https://media.wired.com/photos/5bcea2642eea7906bba84c67/master/w_2560%2Cc_limit/iphonexr.jpg")
colx2.write("Name: option")
colx2.write("Model: option")
colx2.write("Color: option")
colx2.write("Price: option")
col2.title("Data Frame")
df = pd.DataFrame(
    np.random.randn(50, 20),
    columns=('col %d' % i for i in range(20)))

colx3.dataframe(df)
coly1,coly2,coly3=st.columns(3)
chart_data = pd.DataFrame(
     np.random.randn(50, 2),
     columns=["a", "b"])


labels = 'Yes', 'No'
sizes = [70, 30]
explode = (0.1, 0)
coly1.write('Pie Chart')
fig1, ax1 = plt.subplots()
fig1.set_facecolor('#0D1117')
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90,textprops={'color':"w"})
ax1.axis('equal')

coly1.pyplot(fig1)
coly2.write('Bar Chart')
coly2.bar_chart(chart_data,width=500, height=540)
coly3.write('Map data')
d={'latitude':[34.053570000000036],'longitude':[-118.24544999999995]}
data_of_map = pd.DataFrame(d)
coly3.map(data_of_map)      #-118.24544999999995 34.053570000000036

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import pickle
from urllib.request import urlopen
import keras

import tensorflow as tf



t=Tokenizer(num_words=50000,lower=True)
x=t.texts_to_sequences(['my sit down bruce willis i do n’t even remember movie fell love iphone mic attachment in middle interview asked assistant go pick one up he like kid very real excited that ’s i remember', '   i respectfully disagree bitcoin digital gold   ust digital cash i want buy iphone   stock',' that u put google photos cuz iphone gallery i know ur ex got dumped cuz always love get that ’s happened u lie ppl i coming u do n’t care ur girl',' loved tip too make ig ads look simplistic native possible do n’t go super stylized 15 secs less personally i love amateur style ads shot iphone   look sound like product review','   i want buy iphone','   i want buy iphone','   i want buy iphone','   i want buy iphone','    i want buy iphone',' well i break buy new iphone the twitter app version i longer shows comments error message my iphone 6 perfectly good otherwise way i want twitter w/o comments','   i use encounter+ mac iphone plan i get ipad one time buy cheap subscription local hosting local network abroad want pretty easy work bit curve it ’s super helpful dm',' if want sugar daddy buy iphone buy nicki minaj cases blouses i send pictures i still live you',' someone send cash want buy iphone xr already', 'winsforsure' ' i want need new iphone i do n’t want buy money', 'LannyTv' '   i want buy iphone gib quick alpha', 'callmeshazzam' '   i do n’t know i ’m helping mansplaining here   iphone ’s photo edit mirror symmetry switch option photo edit love shirt  ', 'bigwin_metawin' '   hi chita hope win u. i want buy new phone kinda old ca nt update twitr bec using iphone 6 thankyou', 'slaytalia4pres' " okay i changed mind it 's iphone 12 i want buy 430k", 'StrawberryDiar3' '   i want buy i goinh use iphone business', 'RRichPhotos' ' do n’t get samsung phone get iphone oh i love iphone', 'myfriendblondie' ' someone buy nft i want buy iphone nft', 'mrdrew1988' ' miss i do n’t even self follow every day i thank god people i follow like post could show feed hear i love iphone', 'Shkrepsja' ' might tell dad want use money scholarship buy new iphone se', 'AceclothingNg' ' i love   joules oh do n’t notifications appear iphone very frustrating called smart device joule chefsteps', 'HarpSilver' '   perfect color iphone me i want buy it subscribe', 'folyqe' '    i do n’t what ’s happening samsung think want people buy iphone i really miss samsung note 4 7 days a company it ’s thing daring innovative ugly copycat', '__phorcys' '   good morning please i want buy iphone fast charger', 'itsmearrunima' "    i used gphotos since back pixel 2 days i love luckily i need g1 yet i 'm 5 gb i 've used gdrive forever also installed iphone 12 makes sharing easier say", 'TruvybeR' ' speaking i m gon na look iphone 12/13 mini prices want buy one', 'official_ganja' ' buy laptop want iphone please stop purchasing hahhaahahhahahahahaha', '_MrJasonW' '    thanks love android user know thing iphone', 'nofixedrange' ' i do n’t think oneplus10pro bcz i already use iphone since 2015   i love iphones', 'Nova_Lee4444' '   put someone never wants rent iphone buy iphone installments please let continue buy phone outright day 1 keep long i want', 'JustAGalAndADog' ' like iphone trending shut love huawei', 'danielrao007' '   i want buy iphone xr 170k meettolaawoska', 'chasejoynbliss' ' watching bbl videos considering vlogging hm i love talking lol sometimes little much hm decisions decisions i wonder i ’ll need camera cause pictures i take iphone do n’t give it ’s supposed give', 'f2fmuti' '   i would love try iphone any plans android pls', 'wheniminurarms' '   i iphone   another way listen show i would love 2 support you  ', '00ocIock_' '   i want buy iphone 13 dreams', 'eddypham' "   that 's i love iphone 13 mini i need anything larger", 'iam_Ozymandias_' ' want buy new iphone airpods daddy yankee tour tickets kpop albums tom ford ’s lost cherry i ’m fucking poor']) # pro
x=sequence.pad_sequences(x,maxlen=250)

p=model.predict(x)
pred=np.argmax(p,axis=1)
print(pred)
if pred[0]==1:
  print('yes')
elif pred[0]==0:
  print('no')
print('done')	
if __name__ == '__main__':
	main()

In [ ]:
!ngrok authtoken 252TziGYBE8PwNqSDEOvAreeOip_3rrsk8oD3LyJPnttED7DD

In [ ]:
!ngrok http 8080 -host-header="localhost:8080"


In [ ]:
!streamlit run app.py &>/dev/null&


In [ ]:
from pyngrok import ngrok
!nohup streamlit run app.py &
url = ngrok.connect(port = '8501')
print(url)

In [ ]:
ngrok.kill()


In [ ]:
!npm install -g localtunnel


In [ ]:
!lt --port 8888


In [ ]:

t=Tokenizer(num_words=50000,lower=True)
x=t.texts_to_sequences(['I love Iphone']) # pro
x=sequence.pad_sequences(x,maxlen=250)

p=model.predict(x)
print(p)
pred=np.argmax(p,axis=1)
print(pred)
if pred[0]==1:
  print('yes')
elif pred[0]==0:
  print('no')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install pyngrok
!pip install streamlit

In [ ]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 6.7 MB/s 
     |████████████████████████████████| 16.7 MB 36.2 MB/s 
     |████████████████████████████████| 6.3 MB 51.1 MB/s 


# Run project

In [5]:
!pip install pyngrok
!pip install streamlit
!pip install geopandas
!pip install tweepy --upgrade
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob

from keras.backend import manual_variable_initialization 
manual_variable_initialization(True)

import nltk
import spacy
nlp=spacy.load("en_core_web_sm")
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer


# Twitter library
import tweepy
import requests
import json
import tensorflow
from numpy.random import seed
seed(0)
tensorflow.random.set_seed(0)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:

from pyngrok import ngrok 
!ngrok authtoken 252TziGYBE8PwNqSDEOvAreeOip_3rrsk8oD3LyJPnttED7DD
!nohup streamlit run /content/drive/MyDrive/FYP/main.py & 

url = ngrok.connect(port = 8501)
url #generates our URL
print(url)
!streamlit run --server.port 80 /content/drive/MyDrive/FYP/main.py >/dev/null #used for starting our server

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
nohup: appending output to 'nohup.out'
NgrokTunnel: "http://dfca-34-141-234-222.ngrok.io" -> "http://localhost:80"
2022-04-09 11:38:04.686 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
2022-04-09 11:38:25.674966: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/usr/local/lib/python3.7/dist-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/usr/local/lib/python3.7/dist-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
2022-04-09 11:40:05.942 Trac

In [ ]:
!pip install tweepy --upgrade

     |████████████████████████████████| 77 kB 2.5 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.12.1 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.32.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
google-colab 1.0.0 requires tornado~=5.1.0; python_version >= "3.0", but you have tornado 6.1 which is i

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Send DM

In [18]:
import tweepy
  
# assign the values accordingly
consumer_key='vtpCktz8FTXUKEqbrmQyfitZ2'
consumer_secret='FDbBJ3gTQPvv2lA9jT23DDbnDJipV4NjmfEZMobi8Xl7zJO2L1'
access_token='1287085799339462656-yMJ0c9NzCgbwu7sd9QEVjyXUKkWieS'
access_token_secret='OPq0IiDTyGmhziS0EUNyYbupdr492tuB848rtGT8BxzLS'

# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  
# set access to user's access key and access secret 
auth.set_access_token(access_token, access_token_secret)
  
# calling the api 
api = tweepy.API(auth)
  
# ID of the recipient
  
# text to be sent
text = "This is a Direct Message."
user=api.get_user(screen_name="AayushAleron")
ID= user.id_str
# sending the direct message
direct_message = api.send_direct_message(ID, "test")
  
# printing the text of the sent direct message
print(direct_message.message_create['message_data']['text'])

test


In [ ]:
# import the moduleimporttweepy  # assign the values accordinglyconsumer_key=""consumer_secret=""access_token=""access_token_secret=""  # authorization of consumer key and consumer secretauth=tweepy.OAuthHandler(consumer_key, consumer_secret)  # set access to user's access key and access secret auth.set_access_token(access_token, access_token_secret)  # calling the api api=tweepy.API(auth)  # the screen name of the userscreen_name="geeksforgeeks"  # fetching the useruser=api.get_user(screen_name)  # fetching the IDID= user.id_str